In [31]:
import heapq
import random
import networkx as nx
import numpy as np
import math
import csv
from sys import maxsize


In [32]:
def gen_prufer(degree_constrained, n):
    result = []
    candidates = list(range(n))
    while len(result) < n:
        # Lấy một số ngẫu nhiên từ danh sách candidates
        x = random.choice(candidates)
        # Nếu số này chưa xuất hiện quá m lần trong danh sách kết quả, thêm số này vào danh sách kết quả
        if result.count(x) < degree_constrained - 1:
            result.append(x)
        # Nếu số này đã xuất hiện quá m lần, loại bỏ số này khỏi danh sách candidates
        if result.count(x) == degree_constrained - 1:
            candidates.remove(x)
    return result

In [33]:
# Đọc dữ liệu từ tệp CSV và lưu trữ nó trong danh sách
def get_distance_table(file_path):
    records = []
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        # next(reader) # Skip header row
        for row in reader:
            records.append(row)

    # Tạo từ điển để lưu trữ khoảng cách giữa các điểm
    distances_table = {}

    # Tính toán khoảng cách giữa các điểm và lưu trữ kết quả vào từ điển
    for i in range(len(records)):
        for j in range(i + 1, len(records)):
            point1 = (float(records[i][1]), float(records[i][2]))
            point2 = (float(records[j][1]), float(records[j][2]))
            distance = math.dist(point1, point2)
            key = (int(records[i][0]), int(records[j][0]))
            distances_table[key] = distance

    # In ra từ điển khoảng cách
    return distances_table


In [34]:
# distances_table = get_distance_table("")
# a = (1, 0)
# if a in distances_table:
#     print(distances_table[a])
# if a[::-1] in distances_table:
#     print(distances_table[a[::-1]])

In [35]:
def get_distance(edge, distances_table):
    if edge in distances_table:
        return distances_table[edge]
    if edge[::-1] in distances_table:
        return distances_table[edge[::-1]]

In [36]:
def compute_degrees(edges):
    degrees = {}
    for edge in edges:
        for vertex in edge:
            if vertex not in degrees:
                degrees[vertex] = 0
            degrees[vertex] += 1
    return degrees


In [37]:
def calculate_fitness(prufer_sequence, degree_constrained, distances_table):
    tree = nx.from_prufer_sequence(prufer_sequence)
    edges = tree.edges
    degrees = list(compute_degrees(edges).values())
    # print(degrees)

    # Check if the degrees more than target degrees
    if any(x > degree_constrained for x in degrees):
        return 9999999

    cost = 0
    for edge in edges:
        cost = cost + get_distance(edge, distances_table)
    return cost

In [38]:
def crossover(parent1, parent2, crossover_rate=1):
    offspring1 = parent1[:]
    offspring2 = parent2[:]
    if random.random() < crossover_rate:
        crossover_point = random.randint(1, len(parent1) - 2)
        offspring1[crossover_point:], offspring2[crossover_point:] = offspring2[crossover_point:], offspring1[
                                                                                                   crossover_point:]
    return offspring1, offspring2


In [39]:
def mutate(individual, mutation_rate=0.1):
    if random.random() < mutation_rate:
        idx1, idx2 = random.sample(range(len(individual)), 2)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual


def mutate_pop(population, mutation_rate):
    new_pop = []
    for individual in population:
        if random.random() < mutation_rate:
            idx1, idx2 = random.sample(range(len(individual)), 2)
            individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
            new_pop.append(individual)
    return new_pop

In [40]:
# def fix_off(offspring, degree_constrained):
#     counts = {}
#     used = set()
#     new_numbers = []
#
#     for num in offspring:
#         counts[num] = counts.get(num, 0) + 1
#         if counts[num] > degree_constrained:
#             for i in range(0, len(offspring)):
#                 if i not in used:
#                     new_numbers.append(i)
#                     used.add(i)
#                     break
#         else:
#             new_numbers.append(num)
#
#     return new_numbers

In [41]:
def sort_by_value(dict):
    return sorted(dict.items(), key=lambda x: x[1])


def repair(prufer_sequence, degree_constrained):
    set_ver = set(range(len(prufer_sequence) + 2))
    count = {}
    for ver in set_ver:
        count[ver] = prufer_sequence.count(ver)
    count = dict(sort_by_value(count))

    for i in range(len(prufer_sequence)):
        if count[prufer_sequence[i]] > degree_constrained:
            j = next(iter(count))
            count.update([(prufer_sequence[i], count[prufer_sequence[i]] - 1), (j, count[j] + 1)])
            prufer_sequence[i] = j
            count = dict(sort_by_value(count))
    return prufer_sequence

In [42]:
def get_new_pop(population, population_size, degree_constrained, distances_table):
    fitness_values = [calculate_fitness(individual, degree_constrained, distances_table) for
                      individual in population]
    sorted_fitness = sorted(fitness_values)
    index_dict = {val: idx for idx, val in enumerate(fitness_values)}
    # Lấy danh sách index của population đã được sắp xếp theo fitness
    pop_index = [index_dict[val] for val in sorted_fitness]
    # Lấy index của [population_size] phần tử đầu (có cost nhỏ nhất)
    new_pop_index = pop_index[:population_size]
    new_pop = [population[i] for i in new_pop_index]
    return new_pop

In [43]:
def run_ga(n, degree_constrained, distances_table, population_size=500, crossover_rate=0.8, mutation_rate=0.1,
           max_generations=100):
    population = [gen_prufer(degree_constrained, n - 2) for _ in range(population_size)]
    # print(population)
    for generation in range(max_generations):
        # print(population)
        # Tính một mảng fitness value của population
        fitness_values = [calculate_fitness(prufer_sequence, degree_constrained, distances_table) for
                          prufer_sequence in population]
        # print(fitness_values)
        new_population = []
        while len(new_population) < population_size:
            i, j = np.random.choice(range(population_size), size=2, replace=False,
                                    p=np.array(fitness_values) / sum(fitness_values))
            parent1 = population[i]
            parent2 = population[j]
            offspring1, offspring2 = crossover(parent1, parent2)
            new_population.append(offspring1)
            new_population.append(offspring2)
        for i in range(len(new_population)):
            mutate(new_population[i])
            repair(new_population[i], degree_constrained)
        population = population + new_population
        population = get_new_pop(population, population_size, degree_constrained, distances_table)

    return population

In [44]:
# def run_ga(n, degree_constrained, distances_table, population_size=500, crossover_rate=0.8, mutation_rate=0.1,
#            max_generations=10):
#     population = [gen_prufer(degree_constrained, n - 2) for _ in range(population_size)]
#     # print(population)
#     fitness_values = [calculate_fitness(prufer_sequence, degree_constrained, distances_table) for prufer_sequence in
#                           population]
#     counter, generations, min_cost = 0, 0, min(fitness_values)
#     best_choice = population[fitness_values.index(min(fitness_values))]
#
#     while counter < max_generations:
#         print(population)
#         print(fitness_values)
#         new_population = []
#         while len(new_population) < population_size:
#             i, j = np.random.choice(range(population_size), size=2, replace=False,
#                                     p=np.array(fitness_values) / sum(fitness_values))
#             parent1 = population[i]
#             parent2 = population[j]
#             offspring1, offspring2 = crossover(parent1, parent2)
#             offspring1 = fix_off(offspring1, degree_constrained-1)
#             offspring2 = fix_off(offspring2, degree_constrained-1)
#             new_population.append(offspring1)
#             new_population.append(offspring2)
#         for i in range(len(new_population)):
#             mutate(new_population[i])
#         population = new_population
#         fitness_values = [calculate_fitness(prufer_sequence, degree_constrained, distances_table) for prufer_sequence in
#                           population]
#         if min(fitness_values) < min_cost:
#             counter, min_cost = 0, min(fitness_values)
#             best_choice = population[fitness_values.index(min(fitness_values))]
#         else:
#             counter += 1
#         generations += 1
#     return best_choice, generations


In [45]:
degree_constrained = 2
n = 5
distances_table = get_distance_table("data/7_wi29.csv")

population = run_ga(n, degree_constrained, distances_table)
print(population[0])
print(calculate_fitness(population[0], degree_constrained, distances_table))
# fitness_values = [calculate_fitness(prufer_sequence, degree_constrained, distances_table) for prufer_sequence in
#                           last_generation]
# print(fitness_values)

[2, 0, 1]
7570.410430278543


In [46]:
G = nx.from_prufer_sequence(population[0])
print(G.edges())

[(0, 2), (0, 1), (1, 4), (2, 3)]
